# Installazione e Import (Setup)

In [ ]:
%pip install numpy scipy matplotlib

import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

print("Librerie caricate correttamente.")

# Funzioni di Utilità

In [ ]:
def poly_mul(p1, p2):
    return np.convolve(p1, p2)

def poly_add(p1, p2):
    return np.polyadd(p1, p2)

def poly_sub(p1, p2):
    return np.polysub(p1, p2)

def roots_to_poly(roots):
    if len(roots) == 0: return np.array([1.0])
    return np.poly(roots)

def format_poly_string(coeffs):
    coeffs = np.where(np.abs(coeffs) < 1e-5, 0, coeffs)
    if np.all(coeffs == 0): return "0"

    terms = []
    degree = len(coeffs) - 1
    first_term = True
    
    for i, c in enumerate(coeffs):
        power = degree - i
        if c == 0: continue
        if first_term:
            sign = "" if c > 0 else "-"
            first_term = False
        else:
            sign = " + " if c > 0 else " - "
        
        abs_c = abs(c)
        val_str = f"{abs_c:.4f}"
        if power == 0: z_str = ""
        elif power == 1: z_str = "z"
        else: z_str = f"z^{power}"
            
        terms.append(f"{sign}{val_str}{z_str}")
        
    return "".join(terms)

def print_result(name, num, den):
    num_str = format_poly_string(num)
    den_str = format_poly_string(den)
    width = max(len(num_str), len(den_str))
    padding = 4
    line = "-" * width
    
    print(f"\n{name} =")
    print(f"{' ' * padding}{num_str.center(width)}")
    print(f"{' ' * padding}{line}")
    print(f"{' ' * padding}{den_str.center(width)}")

# Calcolo Impianto Discreto

In [ ]:
def get_discrete_sys_case1(a, T, K):
    if a == 0: return [K*T], [1, -1]
    alpha = np.exp(-a*T)
    return [(K/a)*(1-alpha)], [1, -alpha]

def get_discrete_sys_case2(a, T, K):
    alpha = np.exp(-a*T)
    lam = (K/(a**2)) * (a*T - 1 + alpha)
    beta = (K/(a**2)) * (1 - alpha - a*T*alpha)
    return [lam, beta], [1, -(1+alpha), alpha]

def get_discrete_sys_case3(a, b, T, K):
    alpha = np.exp(-a*T)
    common = (a - b) * (1 - alpha)
    lam = (K/(a**2)) * (common + b*a*T)
    beta = -(K/(a**2)) * (common + b*a*T*alpha)
    return [lam, beta], [1, -(1+alpha), alpha]

def get_Pd_from_input(case, a, b, K, T):
    if case == 1:   return get_discrete_sys_case1(a, T, K)
    elif case == 2: return get_discrete_sys_case2(a, T, K)
    elif case == 3: return get_discrete_sys_case3(a, b, T, K)
    else: return [0], [1]

# Sintesi Controllore

In [ ]:
def calculate_stabilizing_controllers(num_P, den_P, T):
    poles = np.roots(den_P)
    unstable_indices = np.where(np.abs(poles) >= 1.0 - 1e-5)[0]
    
    if len(unstable_indices) == 0:
        return ([0.0], [1.0]), ([1.0], [1.0]), ([1.0], [1.0]), (num_P, den_P), True

    else:
        zeros = np.roots(num_P)
        p_unst = poles[np.abs(poles) >= 1.0 - 1e-5]
        p_stab = poles[np.abs(poles) < 1.0 - 1e-5]
        z_unst = zeros[np.abs(zeros) >= 1.0 - 1e-5]
        z_stab = zeros[np.abs(zeros) < 1.0 - 1e-5]

        A_plus = roots_to_poly(p_unst)
        A_minus = roots_to_poly(p_stab)
        B_plus = roots_to_poly(z_unst)
        B_minus_monic = roots_to_poly(z_stab)

        reconstructed = poly_mul(B_plus, B_minus_monic)
        idx = next((i for i, x in enumerate(num_P) if abs(x) > 1e-10), 0)
        k_gain = num_P[idx] / reconstructed[0] if len(reconstructed)>0 else 1.0
        B_minus = k_gain * B_minus_monic

        nP = len(p_unst)
        nZ = len(z_unst)
        E = max(1, (len(den_P)-1) - (len(num_P)-1))
        deg_den_W = nP + nZ + E - 1

        # Calcolo F(z)
        if nP == 1:     
            p = p_unst[0]
            b_val = np.polyval(B_plus, p)
            f0 = (p ** deg_den_W) / b_val
            F_poly = np.array([f0])
        else:
            M_matrix = []
            b_vector = []
            for p in p_unst:
                rhs = (p ** deg_den_W) / np.polyval(B_plus, p)
                b_vector.append(rhs)
                row = [p**k for k in range(nP)]
                M_matrix.append(row)
            try:
                F_poly = np.linalg.solve(M_matrix, b_vector)[::-1]
            except:
                F_poly = np.linalg.lstsq(M_matrix, b_vector, rcond=None)[0][::-1]

        # Calcolo W(z) e H(z)
        W_num = poly_mul(B_plus, F_poly)
        z_pow_W = np.zeros(deg_den_W + 1); z_pow_W[0] = 1.0
        
        diff_poly = poly_sub(z_pow_W, W_num)
        H_num, remainder = signal.deconvolve(diff_poly, A_plus)
        
        if np.max(np.abs(remainder)) > 1e-4:
            print(f"[WARN] Resto divisione H(z) non nullo: {np.max(np.abs(remainder))}")

        z_pow_nP = np.zeros(nP + 1); z_pow_nP[0] = 1.0
        deg_XY = nZ + E - 1
        z_pow_XY = np.zeros(deg_XY + 1); z_pow_XY[0] = 1.0
        
        M = (A_plus, z_pow_nP)
        N = (poly_mul(B_plus, B_minus), poly_mul(A_minus, z_pow_nP))
        X = (poly_mul(A_minus, F_poly), poly_mul(B_minus, z_pow_XY))
        Y = (H_num, z_pow_XY)

        return X, Y, M, N, False

def compute_Cd_generic(X, Y, M, N, Q, is_stable):
    Qn, Qd = Q
    if is_stable:
        Pn, Pd = N
        num_final = poly_mul(Qn, Pd)
        den_final = poly_sub(poly_mul(Qd, Pd), poly_mul(Pn, Qn))
    else:
        Xn, Xd = X; Yn, Yd = Y
        Mn, Md = M; Nn, Nd = N
        
        num_top = poly_add(poly_mul(Xn, poly_mul(Md, Qd)), poly_mul(Mn, poly_mul(Qn, Xd)))
        den_top = poly_mul(Xd, poly_mul(Md, Qd))
        num_bot = poly_sub(poly_mul(Yn, poly_mul(Nd, Qd)), poly_mul(Nn, poly_mul(Qn, Yd)))
        den_bot = poly_mul(Yd, poly_mul(Nd, Qd))
        
        num_final = poly_mul(num_top, den_bot)
        den_final = poly_mul(den_top, num_bot)

    return num_final, den_final

# Main

In [ ]:
# Setup Parametri
CASE = 3
a_val = -1.0 
b_val = 2.0
T_val = 0.1
K_val = 1.0

print(f"--- Configurazione Caso {CASE} ---")
print(f"a = {a_val}, b = {b_val}, T = {T_val}, K = {K_val}")

# 2. Ottieni Pd(z)
pd_num, pd_den = get_Pd_from_input(CASE, a_val, b_val, K_val, T_val)
print_result("Impianto Pd(z)", pd_num, pd_den)

# 3. Ottini X, Y, M, N per Cd(z)
X, Y, M, N, is_stable = calculate_stabilizing_controllers(pd_num, pd_den, T_val)

# 4. Scegli Q(z)
Q = ([5.0], [1.0])

# 5. Calcola Cd(z)
cd_num, cd_den = compute_Cd_generic(X, Y, M, N, Q, is_stable)

# 6. Output Finale
print_result("Controllore Stabilizzante Cd(z)", cd_num, cd_den)